In [16]:
import sys
sys.path.insert(0,r"C:\Users\pc\Documents")
import numpy as np
import pandas as pd
import csv
from sklearn.model_selection import train_test_split
import class_report as cr
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, accuracy_score, auc, f1_score, log_loss
from sklearn.tree import DecisionTreeClassifier 
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier

In [4]:
## Lectura de datos     
base_centro = pd.read_csv(r'C:\Users\pc\Downloads\BASE_CENTRO.txt', sep=r'¡', engine='python')
base_centro.head()

## Eliminación de variables: 
base_centro2 = base_centro[['CUCONUSE', 'SESUSERV', 'SERVDESC', 'SESUSUSC', 'SESUFEIN','DEPADESC', 'SESUCUSA', 'SESUSAPE','SESUCICL', 'SUSCNITC', 
        #'VECTOR','SESUCATE', 'SESUSUCA','SESULOCA','SESUSAAN','V1', 'V2', 'V3', 'V4', 'V5', 'V6',#'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 
        'TIPIFICACION_CLIENTE', 'SUMA','CANTIDAD','CALIFICACION_VECTOR_SERVICIO',
       #'VECTOR_CUALITATIVO_CONTRATO',#'BANCO_1', 'DIA_PAGO_1', 'DIA_PAGO_2','DIA_PAGO_3', 'FECHA_SUSPENSION','RANGO_EDAD', 'GENERO','PROM_SUSC', 'CANAL_ENVIO'
       'SEGMENTACION', 'REGIONAL','ESTRATO_AGRUPADO','DEPARTAMENTO_AGR','ANTIGUEDAD_DIAS', 'CANAL_PAGO']]

base_centro3 = base_centro2.dropna()

In [5]:
## Group by por nit para calificacion
b4 = base_centro3[['SUSCNITC','CALIFICACION_VECTOR_SERVICIO']]
group_nit = b4.groupby(['SUSCNITC'])['CALIFICACION_VECTOR_SERVICIO'].mean().reset_index()


## Join de la nueva calificacion
base_centro4 = pd.merge(base_centro3, group_nit, how='left', on=['SUSCNITC'])

## Prueba promedio
b5 = base_centro4[['SUSCNITC','CALIFICACION_VECTOR_SERVICIO_x','CALIFICACION_VECTOR_SERVICIO_y']]

### Creacion de la etiqueta de clasificacion
base_centro4['y'] = pd.cut(x=base_centro4['CALIFICACION_VECTOR_SERVICIO_y'],
                            bins=[-1,50,76,100],
                            labels=['No pago','Pago inoportuno','Pago'])

In [9]:
dataTypeSeries = base_centro4.dtypes
dataTypeSeries

cat = base_centro4.select_dtypes(include=['object','category'])
cuanti = base_centro4.select_dtypes(include=['int64','float64'])
  

le = LabelEncoder()
cat = cat.apply(lambda col: le.fit_transform(col))
cat.head(10)

base_centro5=cuanti.join(cat)
print(base_centro5.dtypes)

base_centro5.to_csv(r'C:\Users\pc\Desktop\base_centro_process.csv')

CUCONUSE                            int64
SESUSERV                            int64
SESUSUSC                            int64
SESUCUSA                            int64
SESUCICL                            int64
SUMA                                int64
CANTIDAD                            int64
CALIFICACION_VECTOR_SERVICIO_x      int64
ESTRATO_AGRUPADO                  float64
ANTIGUEDAD_DIAS                     int64
CALIFICACION_VECTOR_SERVICIO_y    float64
SERVDESC                            int32
SESUFEIN                            int32
DEPADESC                            int32
SESUSAPE                            int32
SUSCNITC                            int32
TIPIFICACION_CLIENTE                int32
SEGMENTACION                        int32
REGIONAL                            int32
DEPARTAMENTO_AGR                    int32
CANAL_PAGO                          int32
y                                   int32
dtype: object


In [14]:

base_centro5 = pd.read_csv(r'C:\Users\pc\Desktop\base_centro_process.csv', sep=',', engine='python')


X = base_centro5[[
        #'CUCONUSE', 
        'SESUSERV', 'SERVDESC', 'SESUSUSC','DEPADESC', 'SESUCUSA', 'SESUSAPE','SESUCICL', 'TIPIFICACION_CLIENTE', 
        #'SESUFEIN','SUSCNITC', 'VECTOR','SESUCATE', 'SESUSUCA','SESULOCA','SESUSAAN','RANGO_EDAD', 'GENERO', 
        #'V1', 'V2', 'V3', 'V4', 'V5', 'V6','V7', 'V8', 'V9', 'V10', 'V11', 'V12','REGIONAL','PROM_SUSC', 'CANAL_ENVIO'     
        # 'SUMA','CANTIDAD','CALIFICACION_VECTOR_SERVICIO_x', 'CALIFICACION_VECTOR_SERVICIO_y',
        #'VECTOR_CUALITATIVO_CONTRATO','BANCO_1', 'DIA_PAGO_1', 'DIA_PAGO_2','DIA_PAGO_3', 'FECHA_SUSPENSION', 
       'SEGMENTACION','ESTRATO_AGRUPADO','DEPARTAMENTO_AGR','ANTIGUEDAD_DIAS','CANAL_PAGO']]

y = base_centro5['y']

In [11]:
from datetime import datetime
instanteInicial = datetime.now()
X_ = StandardScaler().fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_, y, test_size=0.2,random_state=321)

classifiers = [
    KNeighborsClassifier(3, n_jobs = -1),
    DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1, n_jobs = -1),
    MLPClassifier(alpha=1, max_iter=500),
    LogisticRegression(multi_class="multinomial", solver="lbfgs", C=5, n_jobs = -1),
    GaussianNB()]

# Logging for Visual Comparison
log_cols=["Classifier", "Accuracy", "Log Loss"]
log = pd.DataFrame(columns=log_cols)

for clf in classifiers:
    clf.fit(X_train, y_train)
    name = clf.__class__.__name__
    
    print("="*30)
    print(name)
    
    print('****Results****')
    train_predictions = clf.predict(X_test)
    acc = accuracy_score(y_test, train_predictions)
    print("Accuracy: {:.4%}".format(acc))
    
    train_predictions = clf.predict_proba(X_test)
    ll = log_loss(y_test, train_predictions)
    print("Log Loss: {}".format(ll))
    
    log_entry = pd.DataFrame([[name, acc*100, ll]], columns=log_cols)
    log = log.append(log_entry)
    
print("="*30)

instanteFinal = datetime.now()
tiempo = instanteFinal - instanteInicial # Devuelve un objeto timedelta
segundos = tiempo.seconds

C:\Users\pc\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int32, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\pc\Anaconda3\lib\site-packages\sklearn\base.py:464: DataConversionWarning: Data with input dtype int32, int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


KNeighborsClassifier
****Results****
Accuracy: 84.3018%
Log Loss: 2.5460058939201855
DecisionTreeClassifier
****Results****
Accuracy: 82.2240%
Log Loss: 0.475675844724175
RandomForestClassifier
****Results****
Accuracy: 76.3165%
Log Loss: 0.6422613498820962
MLPClassifier
****Results****
Accuracy: 81.2135%
Log Loss: 0.47214472976815014
LogisticRegression
****Results****
Accuracy: 77.8129%
Log Loss: 0.5535983807937472
GaussianNB
****Results****
Accuracy: 71.2608%
Log Loss: 1.2786515096969784


In [19]:
## Probamos la Red Neuronal Multicapa tambien pero aunque el accuracy y logloss eran buenas tambien, la realidad al 
## hacer una validacion cruzada nos dejaba a la vista el sobreajuste y poco AUC de los clasificadores en varias repeticiones

In [12]:
kf = StratifiedKFold(n_splits=5,shuffle=False)

accuracy_model = []
# Iteraciones del entrenamiento sobre los kfolds
for train_index, test_index in kf.split(X, y) :
    # Particion train test para validacion cruzada
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    # Entrenamiento de la red MLP
    #model = MLPClassifier(alpha=1, max_iter=500).fit(X_train, y_train) 
    model = KNeighborsClassifier(3).fit(X_train, y_train) 
    accuracy_model.append(cr.class_report(y_true = y_test, 
                                       y_pred = model.predict(X_test),
                                       y_score = model.predict_proba(X_test)))
 
# Print the accuracy    
print(accuracy_model)

[             precision    recall  f1-score  support     pred       AUC
0             0.667804  0.719835  0.692844  17404.0  18760.0  0.868159
1             0.768926  0.772873  0.770895  47731.0  47976.0  0.817844
2             0.607657  0.573021  0.589831  28088.0  26487.0  0.769046
avg / total   0.701457  0.702756  0.701769  93223.0  93223.0  0.839089,              precision    recall  f1-score  support     pred       AUC
0             0.656851  0.680591  0.668510  17404.0  18033.0  0.846505
1             0.744408  0.789231  0.766164  47730.0  50604.0  0.805372
2             0.567582  0.496796  0.529835  28088.0  24585.0  0.736592
avg / total   0.674783  0.680837  0.676726  93222.0  93222.0  0.823281,              precision    recall  f1-score  support     pred       AUC
0             0.681199  0.715566  0.697960  17403.0  18281.0  0.867345
1             0.748444  0.778253  0.763057  47730.0  49631.0  0.797597
2             0.573060  0.516360  0.543234  28087.0  25308.0  0.731166
avg

In [17]:
kf = StratifiedKFold(n_splits=5,shuffle=False)

accuracy_model = []
# Iteraciones del entrenamiento sobre los kfolds
for train_index, test_index in kf.split(X, y) :
    # Particion train test para validacion cruzada
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    # Entrenamiento de la red MLP
    model = MLPClassifier(alpha=1, max_iter=500).fit(X_train, y_train) 
    #model = KNeighborsClassifier(3).fit(X_train, y_train) 
    accuracy_model.append(cr.class_report(y_true = y_test, 
                                       y_pred = model.predict(X_test),
                                       y_score = model.predict_proba(X_test)))
 
# Print the accuracy    
print(accuracy_model)

[             precision    recall  f1-score  support     pred       AUC
0             0.382857  0.003850  0.007623  17404.0    175.0  0.502699
1             0.515178  0.996962  0.679320  47731.0  92368.0  0.506310
2             0.682353  0.016520  0.032258  28088.0    680.0  0.506530
avg / total   0.540845  0.516149  0.358960  93223.0  93223.0  0.637247,              precision    recall  f1-score  support     pred       AUC
0             0.191438  0.995920  0.321145  17404.0  90541.0  0.515179
1             0.453552  0.001739  0.003465  47730.0    183.0  0.499689
2             0.834668  0.074231  0.136337  28088.0   2498.0  0.534023
avg / total   0.519448  0.209189  0.102808  93222.0  93222.0  0.406772,              precision    recall  f1-score  support     pred       AUC
0             0.481707  0.004539  0.008994  17403.0    164.0  0.502745
1             0.551430  0.967735  0.702542  47730.0  83764.0  0.571296
2             0.787774  0.260619  0.391664  28087.0   9292.0  0.618162
avg